**Table of contents**<a id='toc0_'></a>    
- [拡張カルマンフィルタによる自己位置推定](#toc1_)    
- [pybulletの起動](#toc2_)    
- [センサクラスの定義](#toc3_)    
  - [IMUクラス](#toc3_1_)    
  - [GPSクラス](#toc3_2_)    
- [拡張カルマンフィルタクラスの定義](#toc4_)    
- [初期設定](#toc5_)    
- [パラメータの設定](#toc6_)    
  - [移動ロボットのパラメータ](#toc6_1_)    
  - [IMUのパラメータ](#toc6_2_)    
  - [GPSのパラメータ](#toc6_3_)    
  - [拡張カルマンフィルタのパラメータ](#toc6_4_)    
- [拡張カルマンフィルタによる自己位置の推定](#toc7_)    

<!-- vscode-jupyter-toc-config
	numbering=false
	anchor=true
	flat=false
	minLevel=1
	maxLevel=6
	/vscode-jupyter-toc-config -->
<!-- THIS CELL WILL BE REPLACED ON TOC UPDATE. DO NOT WRITE YOUR TEXT IN THIS CELL -->

# <a id='toc1_'></a>[拡張カルマンフィルタによる自己位置推定](#toc0_)

本notebookでは、移動ロボットによる拡張カルマンフィルタを用いた自己位置推定を行います。

- カルマンフィルタ参考サイト
    - [Python で学ぶベイズフィルタとカルマンフィルタ：第 11 章 拡張カルマンフィルタ](https://inzkyk.xyz/kalman_filter/extended_kalman_filters/)
    - [robot_pose_ekf](https://github.com/ros-planning/robot_pose_ekf)
    - [robot_localization](https://github.com/cra-ros-pkg/robot_localization)
    - [世界一分かりやすいカルマンフィルタの理論と導出と実装](https://disassemble-channel.com/the-theory-of-kalman-filter/)
    - [1次元カルマンフィルタ](https://www.kalmanfilter.net/JP/kalman1d_jp.html)
    - [カルマンフィルタってなに？](https://qiita.com/IshitaTakeshi/items/740ac7e9b549eee4cc04)

（pybulletで使用可能な関数がまとめられたマニュアルについては[こちら](https://github.com/bulletphysics/bullet3/blob/master/docs/pybullet_quickstartguide.pdf)を参照してください。）

# <a id='toc2_'></a>[pybulletの起動](#toc0_)

In [1]:
import time
import pybullet
import pybullet_data
physics_client = pybullet.connect(pybullet.GUI) 

pybullet build time: Nov 28 2023 23:45:17


startThreads creating 1 threads.
starting thread 0
started thread 0 
argc=2
argv[0] = --unused
argv[1] = --start_demo_name=Physics Server
ExampleBrowserThreadFunc started
X11 functions dynamically loaded using dlopen/dlsym OK!
X11 functions dynamically loaded using dlopen/dlsym OK!
Creating context
Created GL 3.3 context
Direct GLX rendering context obtained
Making context current
GL_VENDOR=Mesa
GL_RENDERER=llvmpipe (LLVM 15.0.7, 256 bits)
GL_VERSION=4.5 (Core Profile) Mesa 23.2.1-1ubuntu3.1~22.04.2
GL_SHADING_LANGUAGE_VERSION=4.50
pthread_getconcurrency()=0
Version = 4.5 (Core Profile) Mesa 23.2.1-1ubuntu3.1~22.04.2
Vendor = Mesa
Renderer = llvmpipe (LLVM 15.0.7, 256 bits)
b3Printf: Selected demo: Physics Server
startThreads creating 1 threads.
starting thread 0
started thread 0 
MotionThreadFunc thread started


# <a id='toc3_'></a>[センサクラスの定義](#toc0_)

## <a id='toc3_1_'></a>[IMUクラス](#toc0_)

In [2]:
import numpy as np

class IMU:
    def __init__(self, robot_id, sensor_link_idx, init_posture, acc_noise_dist, gyro_noise_dist):
        """
        IMUのクラス
        Parameters
        ----------
        robot_id : int
            ロボットのID
        sensor_link_idx : int
            IMUの取り付け位置のリンクのインデックス
        init_posture : list
            初期姿勢（roll, pitch, yaw）
        acc_noise_dist : float
            加速度センサのノイズの分布
        gyro_noise_dist : float
            角速度センサのノイズの分布
        """
        self.robot_id = robot_id
        self.sensor_link_idx = sensor_link_idx
        self.posture = np.array(init_posture)
        self.acc_noise_dist = acc_noise_dist
        self.gyro_noise_dist = gyro_noise_dist
        self.acc = np.zeros(3)
        self.gyro = np.zeros(3)
        self.velocity_prev = np.zeros(3)

    def update_acc(self, dt):
        """
        加速度を更新する
        """
        # リンクの速度を取得（pybulletでは、加速度を直接取得する関数がないため、速度を取得して時間差分から加速度を計算する）
        link_state = pybullet.getLinkState(self.robot_id, self.sensor_link_idx, computeLinkVelocity=1, computeForwardKinematics=1)
        velocity = np.array(link_state[6])
        self.acc = (velocity - self.velocity_prev) / dt
        self.velocity_prev = velocity

        # ノイズを加える
        self.acc += np.random.normal(0, self.acc_noise_dist, 3)

    def get_acc(self):
        """
        加速度を取得する
        """
        return self.acc
    
    def update_gyro(self):
        """
        角速度を更新する
        """
        # リンクの角速度を取得
        link_state = pybullet.getLinkState(self.robot_id, self.sensor_link_idx, computeLinkVelocity=1, computeForwardKinematics=1)  
        self.gyro = np.array(link_state[7])

        # ノイズを加える
        self.gyro += np.random.normal(0, self.gyro_noise_dist, 3)

    def get_gyro(self):
        """
        角速度を取得する
        """
        return self.gyro

    def update_posture(self, dt):
        """
        姿勢を更新する
        """
        self.update_gyro()
        gyro = self.get_gyro()
        self.posture += gyro * dt

        # 姿勢を-π〜πの範囲に収める
        self.posture = np.mod(self.posture + np.pi, 2*np.pi) - np.pi

    def get_posture(self):
        """
        姿勢を取得する
        """
        return self.posture

## <a id='toc3_2_'></a>[GPSクラス](#toc0_)

In [3]:
class GPS:
    def __init__(self, robot_id, sensor_link_idx, noise_dist):
        """
        GPSのクラス
        Parameters
        ----------
        robot_id : int
            ロボットのID
        sensor_link_idx : int
            GPSの取り付け位置のリンクのインデックス
        noise_dist : float
            ノイズの分布
        """
        self.robot_id = robot_id
        self.sensor_link_idx = sensor_link_idx
        self.noise_dist = noise_dist
        self.position = np.zeros(3)

    def update_position(self):
        """
        GPSの位置を更新する
        """
        # リンクの位置を取得
        link_state = pybullet.getLinkState(self.robot_id, self.sensor_link_idx, computeLinkVelocity=1, computeForwardKinematics=1)
        self.position = np.array(link_state[0])[:2]

        # ノイズを加える
        self.position += np.random.normal(0, self.noise_dist, 2)

    def get_position(self):
        """
        GPSの位置を取得する
        """
        return self.position

ven = Mesa


# <a id='toc4_'></a>[拡張カルマンフィルタクラスの定義](#toc0_)

拡張カルマンフィルタは「運動モデルによって予測された状態」と「センサから得られた観測値」をいい感じの比率で組み合わせることで、状態の推定を行う手法です。拡張カルマンフィルタのクラスを以下に示します。

おおまかな流れは以下の通りです。
1. 初期化
    - `__init__`メソッドで、初期状態の設定を行います。
2. 予測
    - `predict`メソッドで、「運動モデルに基づいて推定された状態`x_pred`」を算出します。
3. 更新
    - `update`メソッドで、「`predict`メソッドで予測された状態`x_pred`」と「センサから得られた観測値`z`」を組み合わせて、「運動モデルの情報と観測値の情報によって推定された状態`x_est`」を算出します。
    - この際、「カルマンゲイン`K`」が「運動モデルの情報と観測値の情報をどの程度の割合で組み合わせるか」を決定する重要なパラメータになります。

- なお、組み合わせの比率は「ノイズの大きさ」や「運動モデルの信頼度」によって変わります。
  - 例えば、`Q`が大きい場合、「運動モデルはノイズが発生しやすい」という情報をカルマンフィルタにあらかじめ教えていることになるため、センサから得られた観測値を信用する割合が大きくなります。
  - 逆に、`R`が大きい場合、「センサはノイズが発生しやすい」という情報をカルマンフィルタにあらかじめ教えていることになるため、運動モデルによる推定を信用する割合が大きくなります。
  - また、「観測値`z`」と「推定された観測値`z_pred`」の差が小さい場合、動作モデルが正しく動作していると考えられるため、運動モデルによる推定を信用する割合が大きくなります。
  - 逆に、「観測値`z`」と「推定された観測値`z_pred`」の差が大きい場合、動作モデルが正しく動作していないと考えられるため、センサから得られた観測値を信用する割合が大きくなります。


<br>

まとめると
- 運動モデルによって状態を推定 
- → センサから状態を取得 
- →「運動モデルによって推定された状態」と「センサから得られた状態」をカルマンゲインを用いて良い比率で組み合わせることで最終的な状態を推定

という流れになります。


In [4]:
class EKFkalmanFilter:
    def __init__(self, init_x, init_P, Q, R, mu):
        """
        2次元平面における二輪作動ロボットのカルマンフィルタのクラス
        Parameters
        ----------
        init_x : numpy.ndarray
            初期状態 [x, y, theta]
        init_P : numpy.ndarray
            初期誤差共分散行列
        Q : numpy.ndarray
            プロセスノイズの共分散行列
        R : dict
            観測ノイズの共分散行列
        mu : float
            摩擦係数
        """
        self.x = init_x
        self.P = init_P
        self.Q = Q
        if "wheel_odom" or "imu"  or "gps" in R.keys():
            self.R = R
        else:
            raise ValueError("R_dict must have 'imu' or 'gps' key")
        self.mu = mu
        
    def predict(self, x, u, dt, P_est):
        """
        予測ステップ（運動モデルによる予測）を行う

        Parameters
        ----------
        x : numpy.ndarray
            現在の状態 [x, y, theta]
        u : numpy.ndarray
            入力 [v, w]
        dt : float
            時間ステップ
        P_est : numpy.ndarray
            現在の誤差共分散行列

        Returns
        -------
        x_pred : numpy.ndarray
            運動モデルによって推定された状態 [x, y, theta]
        """
        x_pred = self.motion_model(x, u, dt)
        jF = self.jacobian_F(x_pred, u, dt)
        P_pred = jF @ P_est @ jF.T + self.Q
        return x_pred, P_pred

    def motion_model(self, x, u, dt):
        """
        2次元平面の二輪作動ロボットの運動モデルにより、「現在の状態x」「入力u」から「状態x_pred」を計算する
        
        Parameters
        ----------
        x : numpy.ndarray
            現在の状態 [x, y, theta]
        u : numpy.ndarray
            入力 [v, w]
        dt : float
            時間ステップ

        Returns
        -------
        x_pred : numpy.ndarray
            運動モデルによって推定された状態 [x, y, theta]
        """
        theta = x[2]
        F = np.array([[1.0, 0.0, 0.0],
                      [0.0, 1.0, 0.0],
                      [0.0, 0.0, 1.0]])
        B = np.array([[dt * np.cos(theta), 0.0],
                      [dt * np.sin(theta), 0.0],
                      [0.0, dt]])
        # x_pred = F @ x + B @ u # 摩擦の影響を考慮しない運動モデル

        # 摩擦の影響を考慮した運動モデル
        v_friction = u[0] - u[0] * self.mu
        w_friction = u[1] - u[1] * self.mu
        u_friction = np.array([v_friction, w_friction])
        x_pred = F @ x + B @ u_friction
        return x_pred

    def jacobian_F(self, x, u, dt):
        """
        2次元平面における二輪作動ロボットの運動モデルのヤコビ行列を計算
        
        Parameters
        ----------
        x : numpy.ndarray
            状態 [x, y, theta]
        u : numpy.ndarray
            入力 [v, w]
        dt : float
            時間ステップ

        Returns
        -------
        jF : numpy.ndarray
            2次元平面における二輪作動ロボットの運動モデルのヤコビ行列
        """
        theta = x[2]
        jF = np.array([[1.0, 0.0, -u[0] * dt * np.sin(theta)],
                         [0.0, 1.0, u[0] * dt * np.cos(theta)],
                         [0.0, 0.0, 1.0]])
        return jF

    def update(self, x_pred, P_pred, sensor_type, z):
        """
        更新ステップ（観測モデルによる更新）を行う

        Parameters
        ----------
        x_pred : numpy.ndarray
            予測状態
        z : numpy.ndarray
            観測値
        sensor_type : str
            センサの種類
        P_pred : numpy.ndarray
            予測誤差共分散行列

        Returns
        -------
        x_est : numpy.ndarray
            更新された状態
        P_est : numpy.ndarray
            更新された誤差共分散行列
        """

        # 観測行列のヤコビ行列を計算
        jH = self.jacobian_H(sensor_type)

        # 観測モデルにより予測された観測値を計算
        z_pred = self.observation_model(x_pred, sensor_type)

        # カルマンゲインの計算
        S = jH @ P_pred @ jH.T + self.R[sensor_type]
        K = P_pred @ jH.T @ np.linalg.inv(S)

        # 状態の更新
        x_est = x_pred + K @ (z-z_pred)

        # 誤差共分散行列の更新
        P_est = (np.eye(3) - K @ jH) @ P_pred
        return x_est, P_est

    def observation_model(self, x_pred, sensor_type):
        """
        観測モデルにより、「予測ステップで推定された状態x_pred」から観測値を計算

        Parameters
        ----------
        x_pred : numpy.ndarray
            予測状態
        sensor_type : str
            センサの種類
        """
        H = self.jacobian_H(sensor_type)
        z_pred = H @ x_pred
        return z_pred

    def jacobian_H(self, sensor_type, z=None):
        """
        観測モデルのヤコビ行列を計算
        Parameters
        ----------
        sensor_type : str
            センサの種類
        z : numpy.ndarray
            観測値（今回はヤコビ行列が固定値であるため使用していないが、採用する観測値によっては、zの値をもとにヤコビ行列を動的に計算することもある）
        
        Returns
        -------
        jH : numpy.ndarray
            観測モデルのヤコビ行列
        """
        if sensor_type == "imu":
            jH = np.array([[0.0, 0.0, 1.0]])
        elif sensor_type == "gps":
            jH = np.array([[1.0, 0.0, 0.0],
                           [0.0, 1.0, 0.0]])
        return jH

    def estimation(self, u, z_dict, dt):
        """
        カルマンフィルタの推定ステップ
        
        Parameters
        ----------
        u : numpy.ndarray
            入力 [v, w]
        z_dict : dict
            観測値の辞書{センサの種類: 観測値}
        dt : float
            時間ステップ

        Returns
        -------
        x_est : numpy.ndarray
            推定された状態 [x, y, theta]
        """
        # 予測ステップ
        x_pred, P_pred = self.predict(self.x, u, dt, self.P)

        # 更新ステップ
        for sensor_type, z in z_dict.items():
            x_est, P_est = self.update(x_pred, P_pred, sensor_type, z)
            x_pred = x_est
            P_pred = P_est
        self.x = x_est
        self.P = P_est

        # thetaを -π から π の範囲に収める
        if self.x[2] > np.pi:
            self.x[2] -= 2 * np.pi
        elif self.x[2] < -np.pi:
            self.x[2] += 2 * np.pi

    def get_position(self):
        """
        現在の状態を取得
        Returns
        -------
        x : float
            現在のx座標
        y : float
            現在のy座標
        theta : float
            現在の角度
        """
        return self.x[0], self.x[1], self.x[2]

# <a id='toc5_'></a>[初期設定](#toc0_)

In [5]:
pybullet.resetSimulation() # シミュレーション空間をリセット
pybullet.setAdditionalSearchPath(pybullet_data.getDataPath()) # pybulletに必要なデータへのパスを追加
pybullet.setGravity(0.0, 0.0, -9.8) # 地球上における重力に設定
time_step = 1./240.
pybullet.setTimeStep(time_step)

#床の読み込み
plane_id = pybullet.loadURDF("plane.urdf")

# ロボットの読み込み
car_start_pos = [0, 0, 0.1]  # 初期位置(x,y,z)を設定
car_start_orientation = pybullet.getQuaternionFromEuler([0,0,0])  # 初期姿勢(roll, pitch, yaw)を設定
car_id = pybullet.loadURDF("../urdf/simple_two_wheel_car.urdf",car_start_pos, car_start_orientation)

# GUIモードの際のカメラの位置などを設定
camera_distance = 7.0
camera_yaw = 180.0 # deg
camera_pitch = -90.1 # deg
camera_target_position = [0.0, 0.0, 0.0]
pybullet.resetDebugVisualizerCamera(camera_distance, camera_yaw, camera_pitch, camera_target_position)

ven = Mesa


# <a id='toc6_'></a>[パラメータの設定](#toc0_)

各パラメータの設定を行います。

ノイズの標準偏差などを変更すると、拡張カルマンフィルタの精度が変わります。

## <a id='toc6_1_'></a>[移動ロボットのパラメータ](#toc0_)

In [6]:
# 左右の車輪のジョイントのインデックス
RIGHT_WHEEL_IDX = 0
LEFT_WHEEL_IDX = 1

# 車輪に関する定数
WHEEL_THREAD = 0.325 # 車輪の間隔(「simple_two_wheel_car.urdf」の車輪の間隔と一致させる)

# 入力uのノイズ（値によって、拡張カルマンフィルタの位置推定精度が変わる）
V_NOISE = 0.5 # 並進速度のノイズ
W_NOISE = np.deg2rad(5.0) # 回転速度のノイズ
U_NOISE = np.array([V_NOISE, W_NOISE])

# 並進速度、回転速度のスライダーを作成
linear_vel_slider = pybullet.addUserDebugParameter("linear_velocity", -10, 10, 0)
angular_vel_slider = pybullet.addUserDebugParameter("angular_velocity", -10, 10, 0)

## <a id='toc6_2_'></a>[IMUのパラメータ](#toc0_)

In [7]:
IMU_LINK_IDX = 5
ACC_NOISE_DIST = 0.1 # imuに発生する加速度のノイズの標準偏差（今回の拡張カルマンフィルタによる推定では、加速度の情報は使用していない）
GYRO_NOISE_DIST = np.deg2rad(5.0) # imuに発生するジャイロのノイズの標準偏差（値によって、拡張カルマンフィルタの位置推定精度が変わる）

## <a id='toc6_3_'></a>[GPSのパラメータ](#toc0_)

In [8]:
GPS_LINK_IDX = 5
GPS_NOISE_DIST = 1.0 # gpsに発生するノイズの標準偏差（値によって、拡張カルマンフィルタの位置推定精度が変わる）

## <a id='toc6_4_'></a>[拡張カルマンフィルタのパラメータ](#toc0_)

In [9]:
# プロセスノイズの共分散行列（具体的には、摩擦によるノイズや、「入力ノイズU_NOISE」によるノイズなどを考慮。真のノイズと近いほど、推定精度が向上する）
# ※ 実際にどの程度のノイズが発生するかは分からないこともあるので、試行錯誤的に設定することも多い
Q = np.array([[0.1, 0.0, 0.0], # x方向のノイズの標準偏差
              [0.0, 0.1, 0.0], # y方向のノイズの標準偏差
              [0.0, 0.0, np.deg2rad(1.0)] # 角度のノイズの標準偏差
            ]) ** 2

# 観測ノイズの共分散行列（具体的には、「IMUのノイズGYRO_NOISE_DIST」や「GPSのノイズGPS_NOISE_DIST」などを考慮。真のノイズと近いほど、推定精度が向上する）
# ※ 実際にどの程度のノイズが発生するかは分からないこともあるので、試行錯誤的に設定することも多い
R = {"imu": np.array([[np.deg2rad(5.0)]]),  # imuのyaw角のノイズの標準偏差
     "gps": np.array([[1.0, 0.0], # gpsのx方向のノイズの標準偏差
                      [0.0, 1.0]]) # gpsのy方向のノイズの標準偏差
    }

MU = 0.95 # 摩擦係数（運動モデルにおいて、摩擦を考慮するための係数）

# 推定位置を可視化するための、球体オブジェクトを生成（motion modelのみによる位置推定は赤色、GPSのみによる位置推定は青色、拡張カルマンフィルタによる位置推定は緑色）
motion_model_visual_id = pybullet.createVisualShape(pybullet.GEOM_SPHERE, radius=0.1, rgbaColor=[1,0,0,1])
motion_model_id = pybullet.createMultiBody(0, -1, motion_model_visual_id, [0.0, 0.0, 0.0], useMaximalCoordinates=True)
ekf_visual_id = pybullet.createVisualShape(pybullet.GEOM_SPHERE, radius=0.1, rgbaColor=[0,1,0,1])
ekf_id = pybullet.createMultiBody(0, -1, ekf_visual_id, [0.0, 0.0, 0.0], useMaximalCoordinates=True)

# <a id='toc7_'></a>[拡張カルマンフィルタによる自己位置の推定](#toc0_)

シミュレーションを実行すると、拡張カルマンフィルタによってロボットの自己位置が推定されます。

また、画面上に「真のロボットの位置」と「運動モデルのみによって推定された位置」と「拡張カルマンフィルタによって推定された位置」が表示されます。

In [ ]:
import sys
import random

# debug用の描画を初期化
pybullet.removeAllUserDebugItems()

# ロボットを初期位置にセット
car_start_x = 0.0
car_start_y = -2.5
car_start_theta = 0.0
car_start_pos = [car_start_x, car_start_y, car_start_theta]
car_start_orientation = pybullet.getQuaternionFromEuler([0.0, 0.0, car_start_theta])
pybullet.resetBasePositionAndOrientation(car_id, car_start_pos, car_start_orientation)

# クIMUラスのインスタンス化
imu_init_posture = [car_start_x, car_start_y, car_start_theta]
imu = IMU(car_id, IMU_LINK_IDX, imu_init_posture, ACC_NOISE_DIST, GYRO_NOISE_DIST)

# GPSクラスのインスタンス化
gps = GPS(car_id, GPS_LINK_IDX, GPS_NOISE_DIST)

# 拡張カルマンフィルタに関する設定 ===========================================================
# 初期値
init_x = np.array([car_start_x, car_start_y, car_start_theta]) # 初期状態
init_P = np.array([[0.1, 0.0, 0.0], # x方向の初期誤差共分散
                   [0.0, 0.1, 0.0], # y方向の初期誤差共分散
                   [0.0, 0.0, np.deg2rad(1.0)]]) ** 2 # 角度の初期誤差共分散
ekf = EKFkalmanFilter(init_x, init_P, Q, R, MU)
# =============================================================================================

# motion modelのみによる位置推定のための変数（拡張カルマンフィルタによる推定値との比較用）
motion_model_x = car_start_x
motion_model_y = car_start_y
motion_model_theta = car_start_theta

# Pybulletの画面上に推定位置を表示するための設定（重くなります）
enable_debug_text = False
replace_debug_text_unique_ids = []

# スライダーの設定された値に応じて、移動ロボットを制御し、オドメトリを計算
while True:
    # スライダーの値を取得
    linear_velocity = pybullet.readUserDebugParameter(0)
    angular_velocity = pybullet.readUserDebugParameter(1)

    # 入力値にノイズを加える
    linear_velocity_noised = linear_velocity + np.random.normal(0, U_NOISE[0], 1)
    angular_velocity_noised = angular_velocity + np.random.normal(0, U_NOISE[1], 1)

    # 並進速度、回転速度から左右の車輪の速度を計算（ノイズを加えた値）
    right_wheel_velocity = linear_velocity_noised + angular_velocity_noised * WHEEL_THREAD / 2
    left_wheel_velocity = linear_velocity_noised - angular_velocity_noised * WHEEL_THREAD / 2

    # 速度を設定
    if linear_velocity == 0.0 and angular_velocity == 0.0:
        pybullet.setJointMotorControl2(car_id, RIGHT_WHEEL_IDX, pybullet.VELOCITY_CONTROL, targetVelocity=0)
        pybullet.setJointMotorControl2(car_id, LEFT_WHEEL_IDX, pybullet.VELOCITY_CONTROL, targetVelocity=0)
    else:
        pybullet.setJointMotorControl2(car_id, RIGHT_WHEEL_IDX, pybullet.VELOCITY_CONTROL, targetVelocity=right_wheel_velocity)
        pybullet.setJointMotorControl2(car_id, LEFT_WHEEL_IDX, pybullet.VELOCITY_CONTROL, targetVelocity=left_wheel_velocity)
    pybullet.stepSimulation()

    # imuのセンサ値を取得
    imu.update_posture(time_step)
    imu_posture = imu.get_posture()
    imu_yaw = np.array(imu_posture[2])

    # GPSのセンサ値を取得
    gps.update_position()
    gps_pos = gps.get_position()

    # ここから、拡張カルマンフィルタの処理 ===========================================================
    # 拡張カルマンフィルタの推定ステップ
    u = np.array([linear_velocity, angular_velocity]) # 入力（拡張カルマンフィルタに与える入力は、ノイズを含まない値を与える）
    z_dict = {"imu": imu_yaw, "gps": gps_pos}
    ekf.estimation(u, z_dict, time_step)
    ekf_x, ekf_y, ekf_theta = ekf.get_position()
    # =============================================================================================

    # 真の位置、オドメトリにより推定された位置、拡張カルマンフィルタにより推定された位置
    true_car_pos, true_car_orientation = pybullet.getBasePositionAndOrientation(car_id)
    true_x = true_car_pos[0]
    true_y = true_car_pos[1]
    true_euler = pybullet.getEulerFromQuaternion(true_car_orientation)
    true_theta = true_euler[2]

    # motion modelのみにより推定された位置
    motion_model_pos = ekf.motion_model(np.array([motion_model_x, motion_model_y, motion_model_theta]), u, time_step)
    motion_model_x = motion_model_pos[0]
    motion_model_y = motion_model_pos[1]
    motion_model_theta = motion_model_pos[2]
    if motion_model_theta > np.pi:
        motion_model_theta -= 2 * np.pi
    elif motion_model_theta < -np.pi:
        motion_model_theta += 2 * np.pi

    # 球体オブジェクトを推定位置に移動
    pybullet.resetBasePositionAndOrientation(motion_model_id, [motion_model_x, motion_model_y, 0.0], pybullet.getQuaternionFromEuler([0.0, 0.0, motion_model_theta]))
    pybullet.resetBasePositionAndOrientation(ekf_id, [ekf_x, ekf_y, 0.0], pybullet.getQuaternionFromEuler([0.0, 0.0, ekf_theta]))

    # 「真の位置」「motion modelのみによる位置推定」「拡張カルマンフィルタによる位置推定」を表示
    # 移動量が少ない場合、「motion modelのみによる位置精度 ≒ EKfによる位置精度」（例：linear_velocity=3.0, angular_velocity=10.0など）
    # 移動量が大効くなるほど、「motion modelのみによる位置精度 < EKfによる位置精度」となっていく（例：linear_velocity=10.0, angular_velocity=3.0など）
    sys.stdout.write("\rTrue: ({:.2f}, {:.2f}, {:.2f}), motion model: ({:.2f}, {:.2f}, {:.2f}), EKF: ({:.2f}, {:.2f}, {:.2f})                   ".format(true_x, true_y, true_theta, motion_model_x, motion_model_y, motion_model_theta, ekf_x, ekf_y, ekf_theta))

True: (3.67, 2.48, -0.43), motion model: (0.54, 4.00, 0.07), EKF: (3.19, 2.91, -0.41)                          

KeyboardInterrupt: 

: 